In [ ]:
# Directory of fruits
import os
directory_path = os.getcwd()
print("My current directory is : " + directory_path)
# define the train and test images path
train_dir = directory_path + '\\inputimages\\train\\'
test_dir = directory_path + '\\inputimages\\test\\'

In [ ]:
# list the image file(s)
for dirname, _, filenames in os.walk(test_dir):
    for filename in filenames:
        if filename.split(".")[-1] == "txt": 
            continue
        print(os.path.join(dirname, filename))

In [ ]:
fruits_name = []
fruits_image = []
fruits_dir = []
i = 0
for subdir in os.listdir(train_dir):
    fruits_dir.append(i) # name of the fruit i.e. label
    i += 1
    for image_filename in os.listdir(train_dir + subdir):
        if image_filename.split(".")[-1] == "txt":
            continue
        fruits_name.append(subdir) # name of the fruit 
        fruits_image.append(subdir + '/' + image_filename) # image of the fruit

In [ ]:
import pandas as pd # data processing, CSV file I/O
train_fruits = pd.DataFrame(fruits_name, columns=["Fruits"])
train_fruits["Fruits Image"] = fruits_image
train_fruits

In [ ]:
# count for each fruit
from collections import Counter
# newData is a dictionary
newData = Counter(train_fruits["Fruits"])
print("Found fruits in the data set and their corresponding count of image(s)")
# newData is dictionary
# newData2 is list of tuple 
newData2 = newData.most_common()
newData2

In [ ]:
# list each fruit
x,y = zip(*newData2)
x,y = list(x),list(y)
print("Found fruits: ", x)
print("Number of Images: ", y)

In [ ]:
# plot graphic for each fruit
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(5,5))
ax= sns.barplot(x=x, y=y, palette=sns.color_palette("BuGn_r", 15)) 
plt.xlabel('Fruits', size = 20) 
plt.ylabel('Number of Fruits', size = 20 )
plt.xticks(rotation = 75)
plt.title('Found fruits in the data set and their number') 
plt.show()

In [ ]:
import cv2
from tensorflow.keras.utils import load_img
plt.figure(figsize=(25,16))
how_many_fruits = len(x)

for i in range(how_many_fruits):
    fruits = train_fruits[train_fruits["Fruits"] == x[i]]["Fruits Image"].values [1]    
    plt.subplot(3,5,i+1)
    img = load_img(train_dir + fruits, target_size=(72, 72))    
    plt.imshow(img)
    plt.title(x[i].upper(), color = "green", fontsize = 15 , fontweight = 600)
    plt.axis("off")

plt.suptitle("Fruits' label(s)", fontsize = 25 , color = "darkred", fontweight = 'bold')
plt.show()

In [ ]:
# open an image
img = load_img(train_dir + "/apple/image_19.jpg")
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
# what is the shape of image?
from tensorflow.keras.utils import img_to_array
array_image = img_to_array(img)
print("Image Shape: ", array_image.shape)

In [ ]:
# show how many type(s) of fruit
from glob import glob
fruitCountUnique = glob(train_dir + '/*' )
# declare numberOfClass i.e. 5 fruits
numberOfClass = len(fruitCountUnique)
print("How many different fruits: ", numberOfClass)

In [ ]:
# import model 
import tensorflow as tf 
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense

#import warnings
import warnings
warnings.filterwarnings('ignore')
model = Sequential()
# convolution layer (1 st)
model.add(Conv2D(
    filters=32,
    kernel_size = (5,5),
    input_shape = (72, 72, 3),
    activation = 'relu'))
# pooling layer (1st)
model.add(MaxPooling2D(pool_size=(2,2)))
# dropout layer (1st)
model.add(Dropout(0.1))

# convolution layer (2nd)
model.add(Conv2D(
    filters=64, 
    kernel_size = (5,5),
    activation = 'relu'))
# pooling layer (2nd)
model.add(MaxPooling2D(pool_size=(2,2)))
# dropout layer (2nd )
model.add(Dropout(0.2))

# convlution layer (3rd)
model.add(Conv2D(
    filters=64,
    kernel_size = (5,5),
    activation = 'relu'))
# pooling layer (3rd)
model.add(MaxPooling2D(pool_size=(2,2)))
# dropout layer (3rd )
model.add(Dropout(0.3))
# flatten layer
model.add(Flatten())
# Dense layer
model.add(Dense(
    units=1024, 
    activation='relu'))
# dropout layer
# model.add(Dropout(0.2))
# output layer
model.add(Dense(
    units=numberOfClass, 
    activation='softmax')) 

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = "rmsprop",
    metrics = ["accuracy"])

In [ ]:

# number of training
epochs = 100  
# get 32 images per training
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale= 1./255, # normalize from 0~255 to 0~1
    shear_range = 0.3,
    horizontal_flip=True,
    zoom_range = 0.3)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size = (72, 72),
    batch_size = batch_size,
    color_mode= "rgb",
    class_mode= "categorical")


In [ ]:

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size = (72, 72),
        batch_size = batch_size,
        color_mode= "rgb",
        class_mode= "categorical")


In [ ]:
# get the shape from train_generator
for data_batch, labels_batch in train_generator:
    print("data_batch (train) shape: ", data_batch.shape)
    print("labels_batch (train) shape: ", labels_batch.shape)
    break
# get the shape from test_generator
for data_batch, labels_batch in test_generator:
    print("data_batch (test) shape: ", data_batch.shape)
    print("labels_batch (test) shape: ", labels_batch.shape)
    break

In [ ]:

hist = model.fit_generator(
        generator = train_generator,
        steps_per_epoch = (train_generator.samples) // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps = (test_generator.samples) // batch_size)
model.save('model.h5')

In [ ]:
hist.history.keys()

In [ ]:
import matplotlib.pyplot as plt1
plt1.figure(figsize=(12, 8))
plt1.plot(hist.history["loss"], label = "Train Loss", color = "black")
plt1.plot(hist.history["val_loss"], label = "Validation Loss", color = "darkred", 
         linestyle="dashed", markeredgecolor = "purple", markeredgewidth = 2)
plt1.title("Model Loss", color = "darkred", size = 13)
plt1.legend()
plt1.show()

In [ ]:
import matplotlib.pyplot as plt2
plt2.figure(figsize=(12, 8))
plt2.plot(hist.history["accuracy"],label = "Train Accuracy", color = "black")
plt2.plot(hist.history["val_accuracy"],label = "Validation Accuracy", color = "darkred", 
         linestyle="dashed",markeredgecolor = "purple", markeredgewidth = 2)
plt2.title("Model Accuracy", color = "darkred", size = 13)
plt2.legend()
plt2.show()